<a href="https://colab.research.google.com/github/albercej/zug/blob/master/stylegan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3 from NVIDIA

**Notes**
* To see the original code from NVIDIA [Check here](https://github.com/NVlabs/stylegan3)
* We are using a pretrained model and fine-tuning on top of it.
* If you come across bugs please post them in [Discord](https://discord.com/invite/awREd7EtMA)

---

If you find this notebook useful, consider signing up for my [Code Sprout Newsletter](https://codesprout.substack.com/welcome) or [Check my links](https://shyambv.bio.link/)

Medium article related to it is mentioned []()

# Setup

In [2]:
!pip install einops ninja gdown

     |████████████████████████████████| 108 kB 18.1 MB/s 


In [3]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
if os.path.isdir('/content/drive/MyDrive/WIP/stylegan3/'):
    %cd '/content/drive/MyDrive/WIP/stylegan3/'
else:
    !git clone https://github.com/bvshyam/stylegan3.git /content/drive/MyDrive/WIP/stylegan3/
    %cd '/content/drive/MyDrive/WIP/stylegan3/'

/content/drive/MyDrive/WIP/stylegan3


If you wanted to download the current dataset execute below code

In [ ]:
#!bash /content/drive/MyDrive/WIP/stargan-v2-download/download.sh afhq-dataset


# Custom Dataset

Execute the dataset_tool.py and provide the source and destination of the input dataset. Also mention the resolution required.

Examples:



```
python dataset_tool.py --source=~/downloads/afhqv2/train/cat --dest=~/datasets/afhqv2cat-512x512.zip
python dataset_tool.py --source=~/downloads/afhqv2/train/dog --dest=~/datasets/afhqv2dog-512x512.zip
python dataset_tool.py --source=~/downloads/afhqv2/train/wild --dest=~/datasets/afhqv2wild-512x512.zip
```




In [9]:
!python dataset_tool.py --source=/content/artimages/ --dest=/content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip --resolution='512x512'

Error: Missing input file or directory: /content/artimages/


In [10]:
#Unzip the transformed data if you wanted to verify
!unzip /content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip -d /content/data/artimages_unzipped

unzip:  cannot find or open /content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip, /content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip.zip or /content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip.ZIP.


# Model training

You can start from a pre-trained model. Below are some of the models from Nvdia



```
stylegan3-t-ffhq-1024x1024.pkl, stylegan3-t-ffhqu-1024x1024.pkl, stylegan3-t-ffhqu-256x256.pkl
stylegan3-r-ffhq-1024x1024.pkl, stylegan3-r-ffhqu-1024x1024.pkl, stylegan3-r-ffhqu-256x256.pkl
stylegan3-t-metfaces-1024x1024.pkl, stylegan3-t-metfacesu-1024x1024.pkl
stylegan3-r-metfaces-1024x1024.pkl, stylegan3-r-metfacesu-1024x1024.pkl
stylegan3-t-afhqv2-512x512.pkl
stylegan3-r-afhqv2-512x512.pkl
```



## Using pre-trainined model

In [11]:

# Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
!python /content/drive/MyDrive/WIP/stylegan3/train.py --outdir=~/training-runs --cfg=stylegan3-r --data=/content/artimages-512x512.zip \
    --gpus=1 --batch=32 --batch-gpu=16 --gamma=6.6 --mirror=1 --kimg=500 --snap=5 \
    --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl

Error: --data: [Errno 2] No such file or directory: '/content/artimages-512x512.zip'


## Finetuning a model

In [12]:
batch_size = 16
batch_gpu_size = 8 # Batch size for GPU

dataset_path = '/content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip'
#resume_from = '/content/drive/MyDrive/WIP/stylegan3/models/stylegan3-t-afhqv2-512x512.pkl'  #Can be the link from nvidia
resume_from ='/content/drive/MyDrive/WIP/stylegan3/models/network-a657-r-512-148863.pkl'


gamma_value = 50.0 #Adjustable R1 regularization weight
snapshot_count = 8

In [13]:

# Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
!python /content/drive/MyDrive/WIP/stylegan3/train.py --outdir=/content/drive/MyDrive/WIP/stylegan3/results --cfg=stylegan3-t \
--data=$dataset_path --gpus=1 --batch=$batch_size --batch-gpu=$batch_gpu_size --gamma=$gamma_value --kimg=1 \
--snap=$snapshot_count --resume=$resume_from

Error: --data: [Errno 2] No such file or directory: '/content/drive/MyDrive/WIP/stylegan3/datasets/artimages-512x512.zip'


In [ ]:
#Visualization - does not work on colab
#!python /content/drive/MyDrive/WIP/stylegan3/visualizer.py

# Generate images

In [14]:
model_path = '/content/drive/MyDrive/WIP/stylegan3/results/00002-stylegan3-t-artimages-512x512-gpus1-batch16-gamma50/network-snapshot-000001.pkl'

In [15]:
#Random generation of images using model created

!python /content/drive/MyDrive/WIP/stylegan3/gen_images.py --outdir=out --trunc=1 --seeds=40 \
    --network=$model_path

Loading networks from "/content/drive/MyDrive/WIP/stylegan3/results/00002-stylegan3-t-artimages-512x512-gpus1-batch16-gamma50/network-snapshot-000001.pkl"...
Traceback (most recent call last):
  File "/content/drive/MyDrive/WIP/stylegan3/gen_images.py", line 143, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/content/drive/MyDrive/WIP/stylegan3/gen_images.py", line 107, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/drive/MyDrive/WIP/st

Check-out other articles [Medium](https://bvshyam.medium.com/)